## 2-1. 네이버 영화 리뷰 크롤링
1. 파이썬 표준 라이브러리 csv를 사용
2. 약 1000건의 데이터를 수집하기 위하여 `review_list`의 길이가 1001보다 작은 동안 크롤링을 수행
3. 수집한 데이터의 `sentence`의 내용이 비어있는 경우, 추가하지 않도록 함
4. `sentence`로 받아온 문장 끝의 공백을 미리 처리함(문제 2-3의 조건)
5. 크롤링 과정에서 실시간으로 페이지가 갱신되어 앞에서 수집한 데이터가 다음 페이지로 넘어가는 경우, 중복된 데이터가 수집되는 것을 방지하기 위하여 한 페이지씩 건너뛰며 데이터를 수집함




In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import csv


page = 1
review_list = [["movie", "sentence", "score"]]

while len(review_list) < 1001:
    response = requests.get("https://movie.naver.com/movie/point/af/list.naver?&page=" + str(page))
    soup = BeautifulSoup(
        response.text,
        'html.parser'
    )

    reviews = soup.find('tbody').find_all('tr')
    for review in reviews:
         movie = review.find('td', {'class': 'title'}).find('a', {'class': 'movie color_b'}).text
         sentence = review.find('td', {'class': 'title'}).text.split('\n')[5]
         score = review.find('td', {'class': 'title'}).find('div', {'class': 'list_netizen_score'}).find('em').text

         if sentence == '':
             pass
         else:
             sentence = sentence.replace(",", "")
             sentence = sentence[:-1]
             review_list.append([movie, sentence, score])
          
    page += 2
    sleep(0.5)

f = open(f'samples.csv','w', encoding='utf-8-sig', newline='')
csvWriter = csv.writer(f)
for i in review_list: 
    csvWriter.writerow(i)

f.close()

## 2-2. 네이버 영화 데이터 셋 제작
1. `"samples.csv"`를 기본 인자로 받아 객체 생성
2. 파일 첫 줄의 header를 제거하고, data만을 읽어 `raw_movie_review[0]`이 첫 번째 한줄 평이 나오도록 함
3. Indexing 결과값이 `(str, str, int)` 형태의 튜플로 나오도록 함
4. `__setitem__()`을 재정의하여 `"수정 권한이 없습니다."` 출력 후, 수정이 불가능하도록 구현
5. Index 값이 전체 데이터의 길이를 넘어가는 경우, `IndexError`를 발생하도록 처리함



## 2-3. 네이버 영화 학습 데이터 셋 제작
1. `RawMoviewReview` 클래스를 상속
2. `int`타입의 `score_threadhold`를 받아 생성하도록 함.
3. Indexing 결과값이 `(str, bool)` 형태의 튜플로 나오도록 함
4. 점수가 `score_threadhold` 이상일 경우 `True`, 미만이면 `False`


In [ ]:
import csv


class RawMovieReview:

    def __init__(self, file_name="samples.csv"): 
        self.__file_name = file_name
        with open('samples.csv', newline='', encoding='utf-8-sig') as f:
            self.__data = csv.reader(f)
            self.__header = next(self.__data)
            self.__dataset = [tuple(row) for row in self.__data]
    
    def __len__(self):
        return len(self.__dataset)
    
    def __getitem__(self, index):
        if index > len(self.__dataset):
          raise IndexError

        return (self.__dataset[index][0], self.__dataset[index][1], int(self.__dataset[index][2]))
    
    def __setitem__(self, key, value):
        return print('수정 권한이 없습니다.')


class MovieReview(RawMovieReview):

    def __init__(self, file_name, score_threadhold: int):
      super().__init__()
      self.__score_threadhold = score_threadhold
    
    def __getitem__(self, index):
        if index > super().__len__():
          raise IndexError

        else:
          if int(super().__getitem__(index)[2]) >= self.__score_threadhold:
              return (super().__getitem__(index)[1], True)
          else:
              return (super().__getitem__(index)[1], False)


raw_movie_review = RawMovieReview()
print(len(raw_movie_review))
print(raw_movie_review[0])
print(type(raw_movie_review[0]))
print(type(raw_movie_review[0][0]), type(raw_movie_review[0][1]), type(raw_movie_review[0][2]))

raw_movie_review[0] = "안뇽"
print(raw_movie_review[0])

print()
movie_review = MovieReview(raw_movie_review, 5)
print(movie_review[0])
print(type(movie_review[0]))
print(type(movie_review[0][0]), type(movie_review[0][1]))

1005
('더 수어사이드 스쿼드', '워너 DC는 절대 영화만들때 간섭하지마라 영화는 무조건 19금으로 만들어라', 8)
<class 'tuple'>
<class 'str'> <class 'str'> <class 'int'>
수정 권한이 없습니다.
('더 수어사이드 스쿼드', '워너 DC는 절대 영화만들때 간섭하지마라 영화는 무조건 19금으로 만들어라', 8)

('워너 DC는 절대 영화만들때 간섭하지마라 영화는 무조건 19금으로 만들어라', True)
<class 'tuple'>
<class 'str'> <class 'bool'>


## 아래의 코드는 위와 동일한 과제에 대하여, pandas 라이브러리를 사용하여 구현한 코드입니다.
#### 클래스 구현시, 기본 인자값을 `"samples_with_pandas.csv"`로 지정하였음
#### 이외의 수행 내용은 같음


In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd


page = 1
movie_list = []
sentence_list = []
score_list = []

while len(movie_list) < 1001:
    response = requests.get("https://movie.naver.com/movie/point/af/list.naver?&page=" + str(page))
    soup = BeautifulSoup(
        response.text,
        'html.parser'
    )

    reviews = soup.find('tbody').find_all('tr')
    for review in reviews:
         movie = review.find('td', {'class': 'title'}).find('a', {'class': 'movie color_b'}).text
         sentence = review.find('td', {'class': 'title'}).text.split('\n')[5]
         score = review.find('td', {'class': 'title'}).find('div', {'class': 'list_netizen_score'}).find('em').text

         if sentence == '':
             pass
         else:
             sentence = sentence.replace(",", "")
             sentence = sentence[:-1]
             movie_list.append(movie)
             sentence_list.append(sentence)
             score_list.append(score)
          
    page += 2
    sleep(0.5)

info = {"movie" : movie_list,"sentence":sentence_list, "score":score_list}
movie_review = pd.DataFrame(info)
movie_review.set_index('movie', inplace = True )

movie_review.to_csv("samples_with_pandas.csv", encoding = "utf-8-sig")


In [ ]:
import csv
import pandas as pd


class RawMovieReview:

    def __init__(self, file_name="samples_with_pandas.csv"): 
        self.__file_name = file_name
        self.__dataset = pd.read_csv(self.__file_name)

    def __len__(self):
        return len(self.__dataset)
    
    def __getitem__(self, index):
        if index > len(self.__dataset):
          raise IndexError

        return tuple(self.__dataset.loc[index])
    
    def __setitem__(self, key, value):
        return print('수정 권한이 없습니다.')


class MovieReview(RawMovieReview):

    def __init__(self, file_name, score_threadhold: int):
      super().__init__()
      self.__score_threadhold = score_threadhold
    
    def __getitem__(self, index):
        if index > super().__len__():
          raise IndexError

        else:
          if super().__getitem__(index)[2] >= self.__score_threadhold:
              return (super().__getitem__(index)[1], True)
          else:
              return (super().__getitem__(index)[1], False)


raw_movie_review = RawMovieReview()
print(len(raw_movie_review))
print(raw_movie_review[0])
print(type(raw_movie_review[0]))
print(type(raw_movie_review[0][0]), type(raw_movie_review[0][1]), type(raw_movie_review[0][2]))

raw_movie_review[0] = "안뇽"
print(raw_movie_review[0])

print()
movie_review = MovieReview(raw_movie_review, 5)
print(movie_review[0])
print(type(movie_review[0]))
print(type(movie_review[0][0]), type(movie_review[0][1]))

1005
('미드소마', '시청각 효과가 볼만했으나 그것 뿐이다. 좋은 평가를 주기엔 부족한 작품. 고어 불쾌한 성적 요소 등 애브젝트 미학의 매력을 살리지도 못했다. 여주인공 대니만큼이나 정신이 불안정하면 감명깊게 볼지도 모르겠다.', 4)
<class 'tuple'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
수정 권한이 없습니다.
('미드소마', '시청각 효과가 볼만했으나 그것 뿐이다. 좋은 평가를 주기엔 부족한 작품. 고어 불쾌한 성적 요소 등 애브젝트 미학의 매력을 살리지도 못했다. 여주인공 대니만큼이나 정신이 불안정하면 감명깊게 볼지도 모르겠다.', 4)

('시청각 효과가 볼만했으나 그것 뿐이다. 좋은 평가를 주기엔 부족한 작품. 고어 불쾌한 성적 요소 등 애브젝트 미학의 매력을 살리지도 못했다. 여주인공 대니만큼이나 정신이 불안정하면 감명깊게 볼지도 모르겠다.', False)
<class 'tuple'>
<class 'str'> <class 'bool'>
